# As-a-Judge Methodology with Small LMs for Screening Studies in Systematic Reviews

- This Colab saves all files to Google Drive but loads all data directly from GitHub. Therefore, you only need a Google Drive account to save the processed outputs.

> ⚠️ **Attention:** To use this notebook, you will need your Hugging Face key (`hf_key`) and authorization from the model owners. Each model on Hugging Face has its own page with instructions on how to obtain usage permission.

```python
# Setup your Hugging Face key here
hf_key = 'Your_HF_KEY'


In [ ]:
hf_key = 'YOU_KEY_HERE'

## Import, Config and Some Functions

### Import main package's

In [1]:
# show if GPU-T4 is running up
!nvidia-smi
print('It is running with GPU-T4')

/bin/bash: line 1: nvidia-smi: command not found
It is running with GPU-T4


In [2]:
# IMPORT MAIN PACKAGES
import pandas as pd
import numpy as np
import os
import re
import pandas as pd

# # mont drive for salve processed datasets
# from google.colab import drive
# drive.mount('/content/drive')

from google.colab import files
from typing import List
import os

# update view config on pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# allow Rust use paraellel on python
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

print('Main Packages Installed')

Main Packages Installed


### Import custom packages

#### Balance dataframe with random_state

In [ ]:
def balance_binary_df(df: pd.DataFrame, col: str, random_state=None):
    """
    Balances a binary DataFrame by randomly selecting the same quantity
    of the majority class, maintaining original indices.

    Parameters
    ----------
    df : pd.DataFrame
        Original DataFrame.
    col : str
        Name of the binary column (string).
    random_state : int, optional
        Seed for reproducibility.

    Returns
    -------
    df_low : DataFrame with the minority class
    df_high : DataFrame with the complete majority class
    df_random_high : Random sample of df_high with the same size as df_low
    df_balanced : Union of df_low + df_random_high shuffled
    """

    # Identify counts
    value_counts = df[col].value_counts()

    # Identify minority and majority class
    low_class = value_counts.idxmin()
    high_class = value_counts.idxmax()

    # Separate dataframes
    df_low = df[df[col] == low_class].copy()
    df_high = df[df[col] == high_class].copy()

    # Random sample of the majority class
    df_random_high = df_high.sample(
        n=len(df_low),
        replace=False,
        random_state=random_state
    )

    # Merge maintaining indices and shuffling
    df_balanced = pd.concat([df_low, df_random_high]).sample(
        frac=1,
        random_state=random_state
    )

    return df_balanced


#### Split Dataframe

In [ ]:
def split_dataframe(df: pd.DataFrame, chunk_size: int):
    """
    Divides a DataFrame into fixed-size chunks,
    preserving original indices.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame to be divided.
    chunk_size : int
        Desired chunk size.

    Returns
    -------
    list[pd.DataFrame]
        List of dataframes with up to chunk_size rows each.
    """
    chunks = []
    for start in range(0, len(df), chunk_size):
        end = start + chunk_size
        chunks.append(df.iloc[start:end])
    return chunks


#### EC2 and EC8 clean-up

In [ ]:
def drop_rows_with_ec2_or_ec8(df, applied_column='Applied Criterios'):
    # if the 'applied_column' is not in the dataframe columns, raise an error
    if applied_column not in df.columns:
        raise ValueError(f"column '{applied_column}' not found in the dataframe")

    def contains_ec2_or_ec8(value):
        # if the value is NaN, return false
        if pd.isna(value):
            return False
        # convert the value to uppercase for case-insensitive comparison
        value_upper = str(value).upper()
        # check if 'EC2' or 'EC8' is present in the value
        return ('EC2' in value_upper) or ('EC8' in value_upper)

    # filter the dataframe to exclude rows where the applied_column contains 'EC2' or 'EC8'
    # reset the index of the filtered dataframe
    df_filtered = df[~df[applied_column].apply(contains_ec2_or_ec8)].reset_index(drop=True)
    return df_filtered

#### Shadow Clone and Drop Columns

In [ ]:
def shadow_clone_and_drop_no_target_columns(df, columns):
    df2 = df.copy(deep=True)
    df2 = df2.drop(columns=columns)
    return df2

#### Normalize Status

In [ ]:
def normalize_status(value: str):
    if isinstance(value, str):
        v = value.strip()

        if v.lower() == "incluidos":
            return "Included"

        if v.lower().startswith("e"):
            return "Excluded"

    return value

### Import original dataset - from github

In [7]:

# import dataset and cache files
url  = "https://github.com/izichtl/small-language-models-on-systematic-reviews/raw/main/MEDEIROS_2015_ORIGINAL_RSL_FILES/medeiros_2015_original_rsl_state_01_result.xls"
filename = "medeiros_2015_original_rsl_state_01_result.xls"

# cache the file
if not os.path.exists(filename):
    print('imported from url')
    df = pd.read_excel(url)
    df.to_csv(filename, index=False)
else:
    print('imported from cache')
    df = pd.read_csv(filename)

print(df.columns)
print('imported')


imported from cache
Index(['Revisores', 'ID', 'CODE', 'STATUS', 'Applied Criterios', 'REASON',
       'CATEGORY', 'SUB-CATEGORY', 'YEAR', 'COUNTRY', 'SOURCE', 'TITLE',
       'AUTHORS', 'ABSTRACT', 'URL', 'Unnamed: 15'],
      dtype='object')
imported


## Pre-Processamento

- Needs import, config and sf.

### Understand dataset and look up for columns with nulls

In [ ]:
print(df.info())
(df.isnull().mean() * 100).sort_values(ascending=False)
df["STATUS"].value_counts()

### Cleaning the columns is not relevant to the experiment.

In [ ]:
# clean-up and clone df to work df
rsl_df = shadow_clone_and_drop_no_target_columns(df, ['Revisores',  'CODE', 'REASON', 'CATEGORY', 'SUB-CATEGORY', 'COUNTRY', 'SOURCE', 'AUTHORS', 'URL', 'YEAR', 'Unnamed: 15'])

### Cleaning the instances with EC2 and EC8, these criteria cannot be decided by the LLM.

- EC2. Estudos duplicados ou repetidos.
- EC8. Artigos que não estão disponíveis gratuitamente para download nos ambientes institucionais do CIn/UFPE ou do IFPB;

In [ ]:
rsl_df = drop_rows_with_ec2_or_ec8(rsl_df)

# show nulls and info after critearia clean up
# print(rsl_df.info())


### Cleaning instances with a null abstract or title, because the LLM uses both to make decisions

In [ ]:
rsl_df = rsl_df.dropna(subset=["ABSTRACT"])
rsl_df = rsl_df.dropna(subset=["TITLE"])

print(rsl_df.loc[rsl_df["ABSTRACT"].isnull(), ["ID", "TITLE", "ABSTRACT"]])
print(rsl_df.loc[rsl_df["TITLE"].isnull(), ["ID", "ABSTRACT", "ABSTRACT"]])

# show nulls and info after critearia clean up
print(rsl_df.info())
(rsl_df.isnull().mean() * 100).sort_values(ascending=False)

### Normalization of the Status and Applied Criteria columns to better understand the pre-processed dataset

In [ ]:

# describes status column
print('Absolut Value')
print(rsl_df["STATUS"].value_counts())
print('Percent')
print(rsl_df["STATUS"].value_counts(normalize=True) * 100)
print('')

print(rsl_df["Applied Criterios"].value_counts())
rsl_df["Applied Criterios"] = rsl_df["Applied Criterios"].str.strip()

print('')
# to confirm, where where the status is "included" and the criteria is NaN

count = rsl_df[
    (rsl_df["STATUS"] == "Included") &
    (rsl_df["Applied Criterios"].isna())
].shape[0]
# print(count)

# transfor Applied Criterios where is NaN to "included"
rsl_df["Applied Criterios"] = np.where(rsl_df["Applied Criterios"].isna(), "included", rsl_df["Applied Criterios"])
print('')

print(rsl_df["Applied Criterios"].value_counts())

### Balancing data to 50% included and 50% excluded

Each entry takes 60 - 90 seconds to be processed into model A and B, so a reduce and balance data is a logical choice to turn de experiment be run in time.

In [ ]:
# balanced with random state to be exact result between experiments
df_balanced_a = balance_binary_df(rsl_df, col='STATUS', random_state=42)
df_balanced_b = balance_binary_df(rsl_df, col='STATUS', random_state=42)

if df_balanced_a.equals(df_balanced_b):
    print("dataframes are equals")

df_balanced = df_balanced_a


### Saving RSL pre-processed-dataset

In [ ]:
# df_balanced.to_csv("rsl-pre-processed-dataset-balanced.csv", index=False)
# files.download("rsl-pre-processed-dataset-balanced.csv")
# print(df_balanced)

### Split dataframe by 100 entrys

In [ ]:

df_in_parts = split_dataframe(df_balanced.sample(frac=1, random_state=42), 100)
print(len(df_in_parts))

## Evaluators

- needs hf key and can run without other code cells

#### Evaluator models and Hugginface packages import

In [ ]:
%pip install -q transformers accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
login(hf_key)

In [ ]:
# evaluators control
ENABLE_PIPE_A = False
ENABLE_PIPE_B = False

#### Evaluator A "Qwen/Qwen3-4B-Instruct-2507" loading

In [ ]:
if ENABLE_PIPE_A:
    classification_text_task_pipe_A = pipeline(
        task="text-generation",
        model="Qwen/Qwen3-4B-Instruct-2507",
        device=0,
        return_full_text=False,
    )


#### Evaluator B "google/gemma-3-4b-it" loading

In [ ]:
if ENABLE_PIPE_B:
  classification_text_task_pipe_B = pipeline(
      task="text-generation",
      model="google/gemma-3-4b-it",
      device=0,
      return_full_text=False,
  )

#### Evaluator Pipeline Functions and Parser

In [ ]:
def extract_fields(text):
    result_match = re.search(r'"result"\s*:\s*"([^"]*)"', text)
    explanation_match = re.search(r'"explanation"\s*:\s*"([^"]*)"', text)

    return {
        "result": result_match.group(1) if result_match else None,
        "explanation": explanation_match.group(1) if explanation_match else None
    }

def avaliador_execute_pipeline(
    df, target_cols, prompt_template, pipe,
    output_col_result="result",
    output_col_explain="explain",
    max_tokens=256
):
    df2 = df.copy(deep=True)

    def safe_str(x):
        return "" if pd.isna(x) else str(x)

    resultados = []
    explanations = []

    for i, (idx, row) in enumerate(df2.iterrows()):
        prompt = prompt_template.format(*[safe_str(row[c]) for c in target_cols])
        print("=== INSTANCE ===")
        print(i)

        output = pipe(prompt)

        print("=== RESPONSE ===")
        # print("Raw output:", output)
        print('------------------')

        s = output[0]['generated_text']

        # ----- FIELD EXTRACTION -----
        parsed = extract_fields(s)

        # Check if both were found
        if parsed["result"] is not None and parsed["explanation"] is not None:
            resultados.append(parsed["result"])
            explanations.append(parsed["explanation"])
        else:
            # If it fails, put the raw string in result and an empty explanation
            resultados.append(s)
            explanations.append("")

        print(parsed)
        print('------------------')
        print("=== RESPONSE-END ===")

    df2[output_col_result] = resultados
    df2[output_col_explain] = explanations

    return df2


#### Evaluator Prompts

##### Prompt v2 - CoT

In [ ]:
evaluator_prompt_v2 = """"
You are a scientific article evaluator. Classify the following article as INCLUDED or EXCLUDED
based on the provided inclusion and exclusion criteria. After classifying, provide a brief explanation
justifying your decision.

To perform the evaluation, follow this step-by-step reasoning:

1. Check whether the article violates any Exclusion Criteria:
• EC1. Is the article written in a language other than English?
• EC3. Does the study not address software requirements?
• EC4. Is the study incomplete, a draft, presentation slides, or just an abstract?
• EC5. Is the study tertiary or a meta-analysis?
• EC6. Does it deal with teaching agile methods?
• EC7. Does it fail to address at least one agile methodology?
• EC9. Does it fail to answer at least one research question?

2. Confirm whether the article meets all Inclusion Criteria:
• IC1. Does it address requirements in software projects using agile methodologies?
• IC2. Is it a study from industry or academia?
• IC3. Is it qualitative or quantitative research?
• IC4. Is it a primary or secondary study?

3. If the article does not violate any Exclusion Criteria and satisfies the Inclusion Criteria, classify it as "INCLUDED". Otherwise, classify it as "EXCLUDED".

4. Provide a brief explanation that references the applied criteria justifying your classification.

---
Title: {0}
-
Abstract: {1}

Instruction:
Respond ONLY with a single valid JSON object in the following format, without any additional text or code blocks:

{{
  "result": "INCLUDED" or "EXCLUDED",
  "explanation": "<brief and clear explanation of your classification>"
}}
"""


#### Execution Evaluator A

In [ ]:
# define the evaluator variable
# check if this is the expected execution
ITEM = "A"
EXECUTION = '3'
if ENABLE_PIPE_A:

    # # google drive directory
    # output_dir = "/content/drive/MyDrive/PLN_DATASETS"

    # # ensure the folder exists
    # os.makedirs(output_dir, exist_ok=True)

    print("total chunks:", len(df_in_parts))

    # loop through all chunks
    for i, df_input in enumerate(df_in_parts, start=1):
        print(f"\n processing chunk {i}/{len(df_in_parts)}...")
        print("size:", len(df_input))

        # use variable b dynamically in the output columns
        result_col = f"RESULT_{ITEM}_{EXECUTION}"
        explain_col = f"EXPLAIN_{ITEM}_{EXECUTION}"

        df_result = avaliador_execute_pipeline(
            df=df_input,
            target_cols=["TITLE", "ABSTRACT"],
            prompt_template=evaluator_prompt_v2,
            pipe=classification_text_task_pipe_A,
            output_col_result=result_col,
            output_col_explain=explain_col,
        )

        # automatic filename based on variable b
        output_filename = f"rsl-pre-processed-dataset_{ITEM.lower()}_{i}_execution_{EXECUTION}.csv"
        output_path = os.path.join(output_dir, output_filename)

        # # save to google drive
        # df_result.to_csv(output_path, index=False)
        # print(f"✔ file saved: {output_path}")

    print("\n done with successsss.")


#### Execution Evaluator B

In [ ]:
# define the evaluator variable b
# check if this is the expected execution
ITEM = "B"
EXECUTION = '1'
if ENABLE_PIPE_B:

    # # google drive directory
    # output_dir = "/content/drive/MyDrive/PLN_DATASETS"

    # # ensure the folder exists
    # os.makedirs(output_dir, exist_ok=True)

    print("total chunks:", len(df_in_parts))

    # loop through all chunks
    for i, df_input in enumerate(df_in_parts, start=1):
        print(f"\n processing chunk {i}/{len(df_in_parts)}...")
        print("size:", len(df_input))

        # use variable b dynamically in the output columns
        result_col = f"RESULT_{ITEM}_{EXECUTION}"
        explain_col = f"EXPLAIN_{ITEM}_{EXECUTION}"

        df_result = avaliador_execute_pipeline(
            df=df_input,
            target_cols=["TITLE", "ABSTRACT"],
            prompt_template=evaluator_prompt_v2,
            pipe=evaluator_prompt_v2,
            output_col_result=result_col,
            output_col_explain=explain_col,
        )

        # automatic filename based on variable b
        output_filename = f"rsl-pre-processed-dataset_{ITEM.lower()}_{i}_execution_{EXECUTION}.csv"
        output_path = os.path.join(output_dir, output_filename)

        # # save to google drive
        # df_result.to_csv(output_path, index=False)
        # print(f"✔ file saved: {output_path}")

    print("\n done with successsss.")


## Post-processing

- can run withouu other cells

### Imports packages used in this section

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import Counter
import pandas as pd
import os

# from google.colab import drive
# drive.mount('/content/drive')

# reforce Rust parallel import
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

### Import Judged Datasets from github

In [6]:
judged_a_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judge-processed-dataset_a_execution_1.csv'
judged_b_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judge-processed-dataset_b_execution_1.csv'
judged_c_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judge-processed-dataset_c_execution_1.csv'

judged_a_filename = 'rsl-judge-processed-dataset_a_execution_1.csv'
judged_b_filename = 'rsl-judge-processed-dataset_b_execution_1.csv'
judged_c_filename = 'rsl-judge-processed-dataset_c_execution_1.csv'

if not os.path.exists(judged_a_filename):
    df_judged_a = pd.read_csv(judged_a_url)
    df_judged_a.to_csv(judged_a_filename, index=False)
else:
    df_judged_a = pd.read_csv(judged_a_filename)

if not os.path.exists(judged_b_filename):
    df_judged_b = pd.read_csv(judged_b_url)
    df_judged_b.to_csv(judged_b_filename, index=False)
else:
    df_judged_b = pd.read_csv(judged_b_filename)

if not os.path.exists(judged_c_filename):
    df_judged_c = pd.read_csv(judged_c_url)
    df_judged_c.to_csv(judged_c_filename, index=False)
else:
    df_judged_c = pd.read_csv(judged_c_filename)

print(f"judged dataset a length: {len(df_judged_a)}")
print(f"judged dataset b length: {len(df_judged_b)}")
print(f"judged dataset c length: {len(df_judged_c)}")


judged dataset a length: 598
judged dataset b length: 598
judged dataset c length: 598


#### To normalize judge response

In [ ]:
def standardize_judge_labels(df, target_column):
    """
    standardizes the judgment column labels.
    """
    df_processed = df.copy()
    valid_labels = ['INCLUDED', 'EXCLUDED']

    non_standard_mask = ~df_processed[target_column].isin(valid_labels)

    if non_standard_mask.sum() > 0:
        df_processed.loc[non_standard_mask, target_column] = 'INCLUDED'

    return df_processed


df_judged_a = standardize_judge_labels(df_judged_a, 'RESULT_JUDGE_A')
df_judged_b = standardize_judge_labels(df_judged_b, 'RESULT_JUDGE_B')
df_judged_c = standardize_judge_labels(df_judged_c, 'RESULT_JUDGE_C')


In [ ]:
judge_final_merged_to_metric = df_judged_a.merge(
    df_judged_b[["ID", "RESULT_JUDGE_B", "EXPLAIN_JUDGEB"]],
    on="ID",
    how="left"
)

judge_final_merged_to_metric = judge_final_merged_to_metric.merge(
    df_judged_c[["ID", "RESULT_JUDGE_C", "EXPLAIN_JUDGEC"]],
    on="ID",
    how="left"
)

# judge_final_merged_to_metric




### Import of Evaluated Dataset from github.

In [10]:
base_evaluated_url = "https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_EVALUATED_DATASETS"

# import a list of files of one execution of evaluator
def get_execution_datasets(base_url, execution):
    """
    return a 6 item chunk list.
    """
    datasets = []

    cache_dir = f"cache_{execution[0]}_{execution[1]}"
    os.makedirs(cache_dir, exist_ok=True)

    model_name = execution[4] if len(execution) > 4 else "unknown"
    print(f"processing: {model_name} - {execution[1]}")
    print("-----------------------")

    for item in [1, 2, 3, 4, 5, 6]:
        url = f"{base_url}/{execution[0]}/{execution[1]}/rsl-pre-processed-dataset_{execution[2]}_{item}_execution_{execution[3]}.csv"
        file_name = f"rsl-pre-processed-dataset_{execution[2]}_{item}_execution_{execution[3]}.csv"
        local_path = os.path.join(cache_dir, file_name)

        try:
            if os.path.exists(local_path):
                df = pd.read_csv(local_path)
            else:
                df = pd.read_csv(url)
                df.to_csv(local_path, index=False)

            datasets.append(df)

        except Exception as e:
            print(f"error getting {url}: {e}")

    return datasets

In [9]:
# Folder and group maps
evaludated_a_folders = [
    ["EVALUATED_A","EXECUTION_1", "a", "1", "qwen"],
    ["EVALUATED_A","EXECUTION_2", "a", "2", "qwen"],
    ["EVALUATED_A","EXECUTION_3", "a", "3", "qwen"],
]

evaludated_b_folders = [
    ["EVALUATED_B", "EXECUTION_1", "b", "1", "gemma"],
    ["EVALUATED_B", "EXECUTION_2", "b", "2", "gemma"],
    ["EVALUATED_B", "EXECUTION_3", "b", "3", "gemma"],
]

# A
evaluator_a_execution_1 = evaludated_a_folders[0]
evaluator_a_execution_1 = get_execution_datasets(base_evaluated_url, evaluator_a_execution_1)
evaluator_a_execution_1 = pd.concat(evaluator_a_execution_1, ignore_index=True)

evaluator_a_execution_2 = evaludated_a_folders[1]
evaluator_a_execution_2 = get_execution_datasets(base_evaluated_url, evaluator_a_execution_2)
evaluator_a_execution_2 = pd.concat(evaluator_a_execution_2, ignore_index=True)

evaluator_a_execution_3 = evaludated_a_folders[2]
evaluator_a_execution_3 = get_execution_datasets(base_evaluated_url, evaluator_a_execution_3)
evaluator_a_execution_3 = pd.concat(evaluator_a_execution_3, ignore_index=True)

# B
evaluator_b_execution_1 = evaludated_b_folders[0]
evaluator_b_execution_1 = get_execution_datasets(base_evaluated_url, evaluator_b_execution_1)
evaluator_b_execution_1 = pd.concat(evaluator_b_execution_1, ignore_index=True)

evaluator_b_execution_2 = evaludated_b_folders[1]
evaluator_b_execution_2 = get_execution_datasets(base_evaluated_url, evaluator_b_execution_2)
evaluator_b_execution_2 = pd.concat(evaluator_b_execution_2, ignore_index=True)

evaluator_b_execution_3 = evaludated_b_folders[2]
evaluator_b_execution_3 = get_execution_datasets(base_evaluated_url, evaluator_b_execution_3)
evaluator_b_execution_3 = pd.concat(evaluator_b_execution_3, ignore_index=True)

processing: qwen - EXECUTION_1
-----------------------
processing: qwen - EXECUTION_2
-----------------------
processing: qwen - EXECUTION_3
-----------------------
processing: gemma - EXECUTION_1
-----------------------
processing: gemma - EXECUTION_2
-----------------------
processing: gemma - EXECUTION_3
-----------------------


### Looking for bad formation json's

In [ ]:
# Looking for bad formation json's

# A
evaluator_a_execution_1_filted = ~evaluator_a_execution_1['RESULT_A_1'].isin(['INCLUDED', 'EXCLUDED'])
print('Instances on group_a/_1 that require post-processing')
print(len(evaluator_a_execution_1_filted) - len(evaluator_a_execution_1))

evaluator_a_execution_2_filted = ~evaluator_a_execution_2['RESULT_A_2'].isin(['INCLUDED', 'EXCLUDED'])
print('Instances on group_a/_3 that require post-processing')
print(len(evaluator_a_execution_2_filted) - len(evaluator_a_execution_2))

evaluator_a_execution_3_filted = ~evaluator_a_execution_3['RESULT_A_3'].isin(['INCLUDED', 'EXCLUDED'])
print('Instances on group_a/_3 that require post-processing')
print(len(evaluator_a_execution_3_filted) - len(evaluator_a_execution_3))

# B
evaluator_b_execution_1_filted = ~evaluator_b_execution_1['RESULT_B_1'].isin(['INCLUDED', 'EXCLUDED'])
print('Instances on group_b/_1 that require post-processing')
print(len(evaluator_b_execution_1_filted) - len(evaluator_b_execution_1))

evaluator_b_execution_2_filted = ~evaluator_b_execution_2['RESULT_B_2'].isin(['INCLUDED', 'EXCLUDED'])
print('Instances on group_b/_2 that require post-processing')
print(len(evaluator_b_execution_2_filted) - len(evaluator_b_execution_2))

evaluator_b_execution_3_filted = ~evaluator_b_execution_3['RESULT_B_3'].isin(['INCLUDED', 'EXCLUDED'])
print('Instances on group_b/_3 that require post-processing')
print(len(evaluator_b_execution_2_filted) - len(evaluator_b_execution_3))


### Merging into on dataset by group

In [ ]:
# Merging Group A
df_evaluated_A_merged = evaluator_a_execution_1.merge(
    evaluator_a_execution_2[['ID', 'RESULT_A_2', 'EXPLAIN_A_2']],
    on='ID',
    how='left'
)
df_evaluated_A_merged = df_evaluated_A_merged.merge(
    evaluator_a_execution_3[['ID', 'RESULT_A_3', 'EXPLAIN_A_3']],
    on='ID',
    how='left'
)

# Merging Group B
df_evaluated_B_merged = evaluator_b_execution_1.merge(
    evaluator_b_execution_2[['ID', 'RESULT_B_2', 'EXPLAIN_B_2']],
    on='ID',
    how='left'
)
df_evaluated_B_merged = df_evaluated_B_merged.merge(
    evaluator_b_execution_3[['ID', 'RESULT_B_3', 'EXPLAIN_B_3']],
    on='ID',
    how='left'
)

print("Evaluator A merged into df_evaluated_A_merged ")
print("Evaluator B merged into df_evaluated_B_merged ")


### Method of Majority Voting Semantic Consensus


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def get_best_explication_by_semantic_sim(explicacoes):
    """
    Receives a list of explanations, filters invalid values, and returns the most representative one.
    """
    # --- PROTECTION 1: Filter NaNs, None, and empty strings ---
    valid_explanations = [
        str(e).strip() for e in explicacoes
        if pd.notna(e) and isinstance(e, str) and len(str(e).strip()) > 0
    ]

    # --- PROTECTION 2: Check if anything remains ---
    if not valid_explanations:
        return "No valid explanation available."

    if len(valid_explanations) == 1:
        return valid_explanations[0]

    try:
        # Calculate embeddings only for valid strings
        emb = model.encode(valid_explanations)

        # Calculate cosine similarity
        sim = cosine_similarity(emb)

        # Sum scores to find the semantic "center"
        scores = sim.sum(axis=1)
        idx = np.argmax(scores)

        return valid_explanations[idx]

    except Exception as e:
        # Safety fallback if the model fails
        return valid_explanations[0]


def apply_majority_voting(
    df: pd.DataFrame,
    cols_result: list,
    cols_explain: list,
    binary_options: list,
    output_result_col: str,
    output_explain_col: str
):
    final_results = []
    final_explanations = []

    for idx, row in df.iterrows():
        # Capture results and explanations from the row
        results = [row[c] for c in cols_result]
        explains = [row[c] for c in cols_explain]

        # --- 1. Determine the majority class ---
        # Filter NaNs in the results so as not to count null votes as a class
        valid_results = [r for r in results if pd.notna(r) and r in binary_options]

        if not valid_results:
            # Extreme case: all models failed/returned NaN
            final_results.append("UNDETERMINED")
            final_explanations.append("No valid results for voting.")
            continue

        cont = Counter(valid_results)
        # Get the most common class
        major_class = cont.most_common(1)[0][0]
        final_results.append(major_class)

        # --- 2. Filter explanations only from the winning class ---
        # Here we ensure that we only pass explanations associated with the winning class
        # And we already perform a pre-check if the explanation is not null
        explicacoes_candidatas = [
            e for r, e in zip(results, explains)
            if r == major_class and pd.notna(e)
        ]

        # --- 3. Select the best explanation (with semantic protection) ---
        melhor_exp = get_best_explication_by_semantic_sim(explicacoes_candidatas)
        final_explanations.append(melhor_exp)

    # Add final columns to the dataframe
    df[output_result_col] = final_results
    df[output_explain_col] = final_explanations

    return df


### Execution of Majority Voting Semantic Consensus

In [ ]:
# It uses majority voting and selects the best explanation based on semantic similarity.

# A
evaluator_a_final_dataset = apply_majority_voting(
    df_evaluated_A_merged,
    cols_result=['RESULT_A_1', 'RESULT_A_2', 'RESULT_A_3'],
    cols_explain=['EXPLAIN_A_1', 'EXPLAIN_A_2', 'EXPLAIN_A_3'],
    binary_options=['EXCLUDED', 'INCLUDED'],
    output_result_col='FINAL_RESULT_A',
    output_explain_col='FINAL_EXPLAIN_A'
)

# B
evaluator_b_final_dataset = apply_majority_voting(
    df_evaluated_B_merged,
    cols_result=['RESULT_B_1', 'RESULT_B_2', 'RESULT_B_3'],
    cols_explain=['EXPLAIN_B_1', 'EXPLAIN_B_2', 'EXPLAIN_B_3'],
    binary_options=['EXCLUDED', 'INCLUDED'],
    output_result_col='FINAL_RESULT_B',
    output_explain_col='FINAL_EXPLAIN_B'
)


# # save datasets on drive
# output_dir = "/content/drive/MyDrive/PLN_DATASETS"
# output_filename_ = f"rsl-evaluator-post-processed_a_final.csv"
# output_path_ = os.path.join(output_dir, output_filename_)

# evaluator_a_final_dataset.to_csv(output_path_, index=False)
# print(f"File A saved: {output_path_}")


# output_dir = "/content/drive/MyDrive/PLN_DATASETS"
# output_filename_ = f"rsl-evaluator-post-processed_b_final.csv"
# output_path_ = os.path.join(output_dir, output_filename_)

# evaluator_b_final_dataset.to_csv(output_path_, index=False)
# print(f"File B saved: {output_path_}")


### Execution of Majority Voting Semantic Consensus on Judges

In [ ]:
# It uses majority voting of Judges.

judge = apply_majority_voting(
    judge_final_merged_to_metric,
    cols_result=['RESULT_JUDGE_A', 'RESULT_JUDGE_A', 'RESULT_JUDGE_A'],
    cols_explain=['EXPLAIN_JUDGEA', 'EXPLAIN_JUDGEB', 'EXPLAIN_JUDGEB'],
    binary_options=['EXCLUDED', 'INCLUDED'],
    output_result_col='FINAL_RESULT',
    output_explain_col='FINAL_EXPLAIN'
)


In [ ]:
judge.columns

# save datasets on drive
# output_dir = "/content/drive/MyDrive/PLN_DATASETS"
# output_filename_ = f"rsl-judges-post-processed_final_decision.csv"
# output_path_ = os.path.join(output_dir, output_filename_)

# judge.to_csv(output_path_, index=False)
print(f"Judges Consul saved: {output_path_}")

## Judge

### Import Packages

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

from google.colab import files
from typing import List
import os

# allow Rust use paraellel on python
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
import pandas as pd
import re

%pip install -q transformers accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
login(hf_key)



### Import e Merge Evaluated Datasets

In [ ]:
url_a  = "https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_FINAL_DATASETS/rsl-evaluator-post-processed_a_final.csv"
filename = "rsl-evaluator-post-processed_a_final.csv"

if not os.path.exists(filename):
    df_final_a = pd.read_csv(url_a)
    df_final_a.to_csv(filename, index=False)
else:
    print('Imported from cache')
    df_final_a = pd.read_csv(filename)

url_b  = "https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_FINAL_DATASETS/rsl-evaluator-post-processed_b_final.csv"
filename = "rsl-evaluator-post-processed_b_final.csv"

if not os.path.exists(filename):
    df_final_b = pd.read_csv(url_b)
    df_final_b.to_csv(filename, index=False)
else:
    print('Imported from cache')
    df_final_b = pd.read_csv(filename)

print(len(df_final_a))
print(len(df_final_b))

In [ ]:
df_post_process_final_merged = df_final_a.merge(
    df_final_b[['ID', 'RESULT_B_1',
       'EXPLAIN_B_1', 'RESULT_B_2', 'EXPLAIN_B_2', 'RESULT_B_3', 'EXPLAIN_B_3',
       'FINAL_RESULT_B', 'FINAL_EXPLAIN_B']],
    on='ID',
    how='left'
)
# df_post_process_final_merged.columns


### Prompt

In [ ]:
judge_prompt_v1 = """"
You are a scientific article evaluation judge. You will receive:
- The title and abstract of a scientific article.
- Two independent evaluations (Decision A and Decision B), each containing a classification
  ("INCLUDED" or "EXCLUDED") and an explanation.

Your task is to analyze the article critically and determine **which decision (A or B) better adheres
to the inclusion and exclusion criteria**, and therefore is the most accurate.

To perform the evaluation, follow this step-by-step reasoning:

1. Check whether the article violates any Exclusion Criteria:
• EC1. Is the article written in a language other than English?
• EC3. Does the study not address software requirements?
• EC4. Is the study incomplete, a draft, presentation slides, or just an abstract?
• EC5. Is the study tertiary or a meta-analysis?
• EC6. Does it deal with teaching agile methods?
• EC7. Does it fail to address at least one agile methodology?
• EC9. Does it fail to answer at least one research question?

2. Confirm whether the article meets all Inclusion Criteria:
• IC1. Does it address requirements in software projects using agile methodologies?
• IC2. Is it a study from industry or academia?
• IC3. Is it qualitative or quantitative research?
• IC4. Is it a primary or secondary study?

3. Evaluate Decision A and Decision B:
• Check which decision correctly applies the exclusion criteria.
• Check which decision correctly applies the inclusion criteria.
• Check which explanation shows better reasoning aligned with the title and abstract.
• Identify incorrect assumptions or mistakes in either A or B.
• Select the decision that is more accurate and better justified.

4. Respond ONLY with a single JSON object in the format:

{{
  "best_decision": "INCLUDED" or "EXCLUDED",
  "justification": "<brief explanation of why the chosen decision is more accurate>"
}}

---
Title: {0}

Abstract: {1}

Decision A:
{2}
Justification A:
{3}

Decision B:
{4}
Justification B:
{5}

Instruction:
Respond ONLY with the JSON object specified above. No additional text or code blocks.
"""


### Extrac fields Method

In [ ]:
def extract_fields_v2(text):
    result_match = re.search(r'"best_decision"\s*:\s*"([^"]*)"', text)
    explanation_match = re.search(r'"justification"\s*:\s*"([^"]*)"', text)

    return {
        "result": result_match.group(1) if result_match else None,
        "explanation": explanation_match.group(1) if explanation_match else None
    }

def avaliador_execute_pipeline_judge(
    df, target_cols, prompt_template, pipe,
    output_col_result="result",
    output_col_explain="explain",
    compare_col_a="FINAL_RESULT_A",
    compare_col_b="FINAL_RESULT_B",
    max_tokens=256
):
    """
    Evaluates line by line: if compare_col_a == compare_col_b => does not execute pipe (repeats result, explain="")
    Otherwise => executes pipe and extracts the fields from the generated text.
    """
    df2 = df.copy(deep=True)

    def safe_str(x):
        return "" if pd.isna(x) else str(x)

    resultados = []
    explanations = []

    for i, (idx, row) in enumerate(df2.iterrows()):
        # Line-by-line comparison, handling NaN: consider equal only if both are NaN or values are equal
        a = row.get(compare_col_a)
        b = row.get(compare_col_b)

        both_na = pd.isna(a) and pd.isna(b)
        equal_vals = (a == b) if not (pd.isna(a) or pd.isna(b)) else False

        if both_na or equal_vals:
            # Repeats the result (from compare_col_a) and empty explanation
            resultados.append("" if pd.isna(a) else a)
            explanations.append("")
            print(f"[{i}] SKIP pipe — {compare_col_a} == {compare_col_b} -> result kept: {a}")
            continue

        # If different -> execute pipeline
        prompt = prompt_template.format(*[safe_str(row[c]) for c in target_cols])
        print(f"=== INSTANCE {i} (will run pipe) ===")
        messages = [{"role": "user", "content": f"{prompt}"}]
        print(messages)

        output = pipe(prompt)

        # Extracting the generated text
        s = None
        # Try to extract the generated string safely
        try:
            # Support different pipe formats
            if isinstance(output, list) and len(output) > 0 and 'generated_text' in output[0]:
                s = output[0]['generated_text']
            elif isinstance(output, dict) and 'generated_text' in output:
                s = output['generated_text']
            else:
                # Fallback: stringify
                s = str(output)
        except Exception as e:
            s = str(output)
            print("Warning: could not get generated_text directly:", e)

        print("=== RESPONSE (raw) ===")
        # print("Raw output:", s)
        print('------------------')

        parsed = extract_fields_v2(s)

        # Check if both were found
        if parsed["result"] is not None and parsed["explanation"] is not None:
            resultados.append(parsed["result"])
            explanations.append(parsed["explanation"])
        else:
            # If it fails, put the raw string in result and empty explanation
            resultados.append(s)
            explanations.append("")

        print("Parsed:", parsed)
        print('------------------')
        print("=== RESPONSE-END ===")

    # Assign results to df2
    df2[output_col_result] = resultados
    df2[output_col_explain] = explanations

    return df2

### Execution

In [ ]:
# control judges execution
ENABLE_JUDGE_1 = False
ENABLE_JUDGE_2 = False
ENABLE_JUDGE_3 = False


In [ ]:
if ENABLE_JUDGE_1:
  judge_pipeline_J1 = pipeline(
      task="text-generation",
      model="Unbabel/M-Prometheus-3B",
      device=0,
      return_full_text=False,
  )

if ENABLE_JUDGE_2:
  judge_pipeline_J2 = pipeline(
      task="text-generation",
      model="GAIR/autoj-bilingual-6b",
      device=0,
      return_full_text=False,
  )

if ENABLE_JUDGE_3:
  judge_pipeline_J3 = pipeline(
      task="text-generation",
      model="grounded-ai/phi3-hallucination-judge-merge",
      device=0,
      return_full_text=False,
  )



### Judges Execution

In [ ]:
ITEM = "A"
EXECUTION = '1'
if ENABLE_JUDGE_1:

    # output_dir = "/content/drive/MyDrive/PLN_DATASETS"
    # import os
    # os.makedirs(output_dir, exist_ok=True)

    df_input = df_post_process_final_merged
    print("Tamanho:", len(df_input))

    result_col = f"RESULT_JUDGE_{ITEM}"
    explain_col = f"EXPLAIN_JUDGE{ITEM}"

    df_result = avaliador_execute_pipeline_judge(
        df=df_input,
        target_cols=["TITLE", "ABSTRACT", 'FINAL_RESULT_A','FINAL_EXPLAIN_A', 'FINAL_RESULT_B', 'FINAL_EXPLAIN_B'],
        prompt_template=judge_prompt_v1,
        pipe=judge_pipeline_J1,
        output_col_result=result_col,
        output_col_explain=explain_col,
        compare_col_a="FINAL_RESULT_A",
        compare_col_b="FINAL_RESULT_B"
    )

    output_filename = f"rsl-judge-processed-dataset_{ITEM.lower()}_execution_{EXECUTION}.csv"
    output_path = os.path.join(output_dir, output_filename)

    df_result.to_csv(output_path, index=False)
    print(f"✔ Arquivo salvo: {output_path}")

    print("\nDone with successsss.")


ITEM = "B"
EXECUTION = '1'
if ENABLE_JUDGE_2:

    # output_dir = "/content/drive/MyDrive/PLN_DATASETS"
    # import os
    # os.makedirs(output_dir, exist_ok=True)

    df_input = df_post_process_final_merged
    print("Tamanho:", len(df_input))

    result_col = f"RESULT_JUDGE_{ITEM}"
    explain_col = f"EXPLAIN_JUDGE{ITEM}"

    df_result = avaliador_execute_pipeline_judge(
        df=df_input,
        target_cols=["TITLE", "ABSTRACT", 'FINAL_RESULT_A','FINAL_EXPLAIN_A', 'FINAL_RESULT_B', 'FINAL_EXPLAIN_B'],
        prompt_template=judge_prompt_v1,
        pipe=judge_pipeline_J2,
        output_col_result=result_col,
        output_col_explain=explain_col,
        compare_col_a="FINAL_RESULT_A",
        compare_col_b="FINAL_RESULT_B"
    )

    output_filename = f"rsl-judge-processed-dataset_{ITEM.lower()}_execution_{EXECUTION}.csv"
    output_path = os.path.join(output_dir, output_filename)

    df_result.to_csv(output_path, index=False)
    print(f"✔ Arquivo salvo: {output_path}")

    print("\nDone with successsss.")

ITEM = "C"
EXECUTION = '1'
if ENABLE_JUDGE_3:

    # output_dir = "/content/drive/MyDrive/PLN_DATASETS"
    # import os
    # os.makedirs(output_dir, exist_ok=True)

    df_input = df_post_process_final_merged
    print("Tamanho:", len(df_input))

    result_col = f"RESULT_JUDGE_{ITEM}"
    explain_col = f"EXPLAIN_JUDGE{ITEM}"

    df_result = avaliador_execute_pipeline_judge(
        df=df_input,
        target_cols=["TITLE", "ABSTRACT", 'FINAL_RESULT_A','FINAL_EXPLAIN_A', 'FINAL_RESULT_B', 'FINAL_EXPLAIN_B'],
        prompt_template=judge_prompt_v1,
        pipe=judge_pipeline_J3,
        output_col_result=result_col,
        output_col_explain=explain_col,
        compare_col_a="FINAL_RESULT_A",
        compare_col_b="FINAL_RESULT_B"
    )

    output_filename = f"rsl-judge-processed-dataset_{ITEM.lower()}_execution_{EXECUTION}.csv"
    output_path = os.path.join(output_dir, output_filename)

    # df_result.to_csv(output_path, index=False)

    print("\nDone with successsss.")


## Evaluation of results

- Can run without others cells

### Import packages used in Evaluation

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from itertools import combinations
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# mont drive for salve processed datasets
# from google.colab import drive
# drive.mount('/content/drive')

### Import Evalutor A and B Datasets from github

In [ ]:

evaluator_final_dataset_a_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_FINAL_DATASETS/rsl-evaluator-post-processed_a_final.csv'
evaluator_final_dataset_b_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_FINAL_DATASETS/rsl-evaluator-post-processed_b_final.csv'

evaluator_final_dataset_a_filename = 'rsl-evaluator-post-processed_a_final.csv'
evaluator_final_dataset_b_filename = 'rsl-evaluator-post-processed_b_final.csv'


if not os.path.exists(evaluator_final_dataset_a_filename):
    evaluator_final_dataset_a = pd.read_csv(evaluator_final_dataset_a_url)
    evaluator_final_dataset_a.to_csv(evaluator_final_dataset_a_filename, index=False)
else:
    evaluator_final_dataset_a = pd.read_csv(evaluator_final_dataset_a_filename)

if not os.path.exists(evaluator_final_dataset_b_filename):
    evaluator_final_dataset_b = pd.read_csv(evaluator_final_dataset_b_url)
    evaluator_final_dataset_b.to_csv(evaluator_final_dataset_b_filename, index=False)
else:
    evaluator_final_dataset_b = pd.read_csv(evaluator_final_dataset_b_filename)

print(f"Evaluator dataset a len: {len(evaluator_final_dataset_a)}")
print(f"Evaluator dataset b len: {len(evaluator_final_dataset_b)}")



#### To merge into a final evaluator dataset

In [ ]:
# merge dataset to calc metric between
evaluator_final_merged_to_metric = evaluator_final_dataset_a.merge(
    evaluator_final_dataset_b[["ID", 'RESULT_B_1','EXPLAIN_B_1', 'RESULT_B_2', 'EXPLAIN_B_2', 'RESULT_B_3', "FINAL_RESULT_B", "FINAL_EXPLAIN_B"]],
    on="ID",
    how="left"
)

evaluator_final_merged_to_metric.columns

### Import Judged Datasets from github

In [ ]:
judged_a_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judge-processed-dataset_a_execution_1.csv'
judged_b_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judge-processed-dataset_b_execution_1.csv'
judged_c_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judge-processed-dataset_c_execution_1.csv'

judged_a_filename = 'rsl-judge-processed-dataset_a_execution_1.csv'
judged_b_filename = 'rsl-judge-processed-dataset_b_execution_1.csv'
judged_c_filename = 'rsl-judge-processed-dataset_c_execution_1.csv'

if not os.path.exists(judged_a_filename):
    df_judged_a = pd.read_csv(judged_a_url)
    df_judged_a.to_csv(judged_a_filename, index=False)
else:
    df_judged_a = pd.read_csv(judged_a_filename)

if not os.path.exists(judged_b_filename):
    df_judged_b = pd.read_csv(judged_b_url)
    df_judged_b.to_csv(judged_b_filename, index=False)
else:
    df_judged_b = pd.read_csv(judged_b_filename)

if not os.path.exists(judged_c_filename):
    df_judged_c = pd.read_csv(judged_c_url)
    df_judged_c.to_csv(judged_c_filename, index=False)
else:
    df_judged_c = pd.read_csv(judged_c_filename)

print(f"Judged dataset a len: {len(df_judged_a)}")
print(f"Judged dataset b len: {len(df_judged_b)}")
print(f"Judged dataset c len: {len(df_judged_c)}")


#### To normalize judge responde

In [ ]:
def standardize_judge_labels(df, target_column):
    """
    Standardizes the judgment column labels.
    """
    # Create a copy to ensure we are modifying a specific instance and avoiding view warnings
    df_processed = df.copy()
    valid_labels = ['INCLUDED', 'EXCLUDED']

    # Identify rows that have non-standard labels (neither INCLUDED nor EXCLUDED)
    non_standard_mask = ~df_processed[target_column].isin(valid_labels)

    # Update values
    # Rationale: To avoid loss of evidence, studies mentioned by the judge in the response
    # (which originally had non-standard labels) were added as 'INCLUDED'.
    if non_standard_mask.sum() > 0:
        df_processed.loc[non_standard_mask, target_column] = 'INCLUDED'

    return df_processed


df_judged_a = standardize_judge_labels(df_judged_a, 'RESULT_JUDGE_A')
df_judged_b = standardize_judge_labels(df_judged_b, 'RESULT_JUDGE_B')
df_judged_c = standardize_judge_labels(df_judged_c, 'RESULT_JUDGE_C')

print(f"Judged dataset a len: {len(df_judged_a)}")
print(df_judged_a['RESULT_JUDGE_A'].value_counts())
print("-" * 30)

print(f"Judged dataset b len: {len(df_judged_b)}")
print(df_judged_b['RESULT_JUDGE_B'].value_counts())
print("-" * 30)

print(f"Judged dataset c len: {len(df_judged_c)}")
print(df_judged_c['RESULT_JUDGE_C'].value_counts())

In [ ]:
judge_final_merged_to_metric = df_judged_a.merge(
    df_judged_b[["ID", "RESULT_JUDGE_B", "EXPLAIN_JUDGEB"]],
    on="ID",
    how="left"
)

judge_final_merged_to_metric = judge_final_merged_to_metric.merge(
    df_judged_c[["ID", "RESULT_JUDGE_C", "EXPLAIN_JUDGEC"]],
    on="ID",
    how="left"
)


### Import Judges Final Decision Dataset

In [ ]:
judged_final_url = 'https://raw.githubusercontent.com/izichtl/small-language-models-on-systematic-reviews/refs/heads/main/PLN_JUDGED_DATASETS/rsl-judges-post-processed_final_decision.csv'


judged_final_filename = 'rsl-judges-post-processed_final_decision.csv'


if not os.path.exists(judged_final_filename):
    judged_final = pd.read_csv(judged_final_url)
    judged_final.to_csv(judged_final_filename, index=False)
else:
    judged_final = pd.read_csv(judged_final_filename)


print(f"Judged Final dataset a len: {len(judged_final)}")



### Create the final dataset of project - RSL_SML_JUDGES

In [ ]:
final_project_dataset = evaluator_final_merged_to_metric.merge(
    judge_final_merged_to_metric[["ID", 'RESULT_JUDGE_A', 'EXPLAIN_JUDGEA', 'RESULT_JUDGE_B', 'EXPLAIN_JUDGEB', 'RESULT_JUDGE_C', 'EXPLAIN_JUDGEC']],
    on="ID",
    how="left"
)

final_project_dataset = final_project_dataset.merge(
    judged_final[["ID", 'FINAL_RESULT', 'FINAL_EXPLAIN',]],
    on="ID",
    how="left"
)


final_project_dataset = final_project_dataset.rename(columns={'EXPLAIN_JUDGEA': 'EXPLAIN_JUDGE_A'})
final_project_dataset = final_project_dataset.rename(columns={'EXPLAIN_JUDGEB': 'EXPLAIN_JUDGE_B'})
final_project_dataset = final_project_dataset.rename(columns={'EXPLAIN_JUDGEC': 'EXPLAIN_JUDGE_C'})
final_project_dataset['STATUS'] = final_project_dataset['STATUS'].str.upper()
final_project_dataset.columns

# save datasets on drive
output_dir = "/content/drive/MyDrive/PLN_DATASETS"
output_filename_ = f"slm-as-a-judge-on-rsl-dataset.csv"
output_path_ = os.path.join(output_dir, output_filename_)

# final_project_dataset.to_csv(output_path_, index=False)
print(f"Judges Consul saved: {output_path_}")



### Methods for calculating and plotting similarity metrics.

#### Merge all results into one dataset

In [ ]:
def consolidar_resultados(dados_entrada):
    """
    Receives a single DataFrame or a list of DataFrames generated by the 'gerar_tabela_concordancia' function.
    Returns a consolidated table formatted for publication.
    """

    # 1. Normalization: Ensures we are dealing with a list, even if only one item is provided
    if isinstance(dados_entrada, pd.DataFrame):
        lista_dfs = [dados_entrada]
    elif isinstance(dados_entrada, list):
        lista_dfs = dados_entrada
    else:
        raise ValueError("The input must be a DataFrame or a list of DataFrames.")

    linhas_finais = []

    for df in lista_dfs:
        # Extracts key rows from the original DF
        df_fleiss = df[df['Métrica'] == "Fleiss' Kappa (Geral)"]
        df_krip = df[df['Métrica'] == "Krippendorff's Alpha (Geral)"]
        df_cohen_media = df[df['Métrica'] == "Cohen's Kappa (Média)"]

        # Extracts only the paired rows (excludes average and general metrics)
        # Identifies rows that contain "Cohen" but NOT "Média"
        df_pares = df[
            df['Métrica'].str.contains("Cohen", na=False) &
            ~df['Métrica'].str.contains("Média", na=False)
        ]

        # Auxiliary variables for general values
        val_fleiss = df_fleiss['Valor'].values[0] if not df_fleiss.empty else '--'
        int_fleiss = df_fleiss['Interpretação'].values[0] if not df_fleiss.empty else '--'

        val_krip = df_krip['Valor'].values[0] if not df_krip.empty else '--'
        int_krip = df_krip['Interpretação'].values[0] if not df_krip.empty else '--'

        val_cohen_med = df_cohen_media['Valor'].values[0] if not df_cohen_media.empty else '--'
        int_cohen_med = df_cohen_media['Interpretação'].values[0] if not df_cohen_media.empty else '--'

        # List to collect annotator names for this group (to create the compound model name)
        anotadores_grupo = set()

        # CASE 1: Only 1 pair (Simple binary comparison, e.g., Human x Model)
        # In this case, general Fleiss and Krip apply to this single row.
        eh_comparacao_binaria = len(df_pares) == 1

        for _, row in df_pares.iterrows():
            # Parse the name: "Cohen's Kappa (B_1 vs B_2)" -> "B_1 x B_2"
            metrica_raw = row['Métrica']
            # Get what's inside the parentheses
            conteudo = metrica_raw.split('(')[1].split(')')[0]
            partes = conteudo.split(' vs ')

            nome_modelo = f"{partes[0]} x {partes[1]}"
            anotadores_grupo.update(partes)

            nova_linha = {
                'Model': nome_modelo,
                'Cohen Kappa': row['Valor'],
                'Interp CK': row['Interpretação'],
                # If binary, fill everything. If group, pairs do not have individual Fleiss/Krip in the standard output
                'Fleiss Kappa': val_fleiss if eh_comparacao_binaria else '--',
                'Interp FK': int_fleiss if eh_comparacao_binaria else '--',
                'Krippendorf': val_krip if eh_comparacao_binaria else '--',
                'Interp Krip': int_krip if eh_comparacao_binaria else '--'
            }
            linhas_finais.append(nova_linha)

        # CASE 2: Group Comparison (>2 annotators)
        # Adds a summary row at the end with the combination of all
        if not eh_comparacao_binaria and len(anotadores_grupo) > 0:
            # Creates compound name: "B_1 x B_2 x B_3"
            nome_grupo = " x ".join(sorted(list(anotadores_grupo)))

            linha_resumo = {
                'Model': nome_grupo,
                'Cohen Kappa': val_cohen_med, # Average of Cohens
                'Interp CK': int_cohen_med,
                'Fleiss Kappa': val_fleiss,
                'Interp FK': int_fleiss,
                'Krippendorf': val_krip,
                'Interp Krip': int_krip
            }
            linhas_finais.append(linha_resumo)

    # Creates the final DataFrame
    df_final = pd.DataFrame(linhas_finais)

    # Reorder columns to ensure the requested layout
    cols_order = [
        'Model',
        'Cohen Kappa', 'Interp CK',
        'Fleiss Kappa', 'Interp FK',
        'Krippendorf', 'Interp Krip'
    ]

    return df_final[cols_order]

#### Calcule Metrics

In [ ]:
def nominal_krippendorff_alpha(data):
    units_cnt = len(data)
    if units_cnt == 0: return 0

    vals = np.unique(data)
    val_map = {v: i for i, v in enumerate(vals)}
    n_cat = len(vals)

    matrix = np.vectorize(val_map.get)(data)
    n_observers = matrix.shape[1]

    coincidence_matrix = np.zeros((n_cat, n_cat))

    for row in matrix:
        counts = np.bincount(row, minlength=n_cat)
        for i in range(n_cat):
            for j in range(n_cat):
                if i == j:
                    coincidence_matrix[i, j] += counts[i] * (counts[i] - 1)
                else:
                    coincidence_matrix[i, j] += counts[i] * counts[j]

    n_total_judgments = units_cnt * n_observers * (n_observers - 1)
    if n_total_judgments == 0: return 0

    sum_diag = np.trace(coincidence_matrix)
    observed_agreement = sum_diag / n_total_judgments

    row_sums = np.sum(coincidence_matrix, axis=1)
    total_sum = np.sum(row_sums)
    expected_agreement = np.sum(row_sums**2) / (total_sum**2)

    if expected_agreement == 1: return 1.0

    alpha = 1 - (1 - observed_agreement) / (1 - expected_agreement)
    return alpha

def gerar_tabela_concordancia(titulo, df, colunas_anotadores, labels_anotadores=None):
    """
    Calcula Cohen (pares), Fleiss e Krippendorff.

    Args:
        titulo (str): Título da análise (usado apenas para log ou controle).
        df (pd.DataFrame): DataFrame com os dados.
        colunas_anotadores (list): Lista com o nome das colunas no DF (ex: ['col_a', 'col_b']).
        labels_anotadores (list, opcional): Lista com os nomes de exibição (ex: ['Humano', 'Modelo']).
                                            Deve ter o mesmo tamanho de colunas_anotadores.
    """

    # Validação de tamanho
    if len(colunas_anotadores) < 2:
        return pd.DataFrame([{"Erro": "Necessário no mínimo 2 colunas para comparação"}])

    # Se labels foram passados, verifica se batem com as colunas
    if labels_anotadores is not None:
        if len(labels_anotadores) != len(colunas_anotadores):
            return pd.DataFrame([{"Erro": f"Número de labels ({len(labels_anotadores)}) diferente do número de colunas ({len(colunas_anotadores)})." }])
        # Cria mapa: Coluna -> Nome Bonito
        mapa_nomes = dict(zip(colunas_anotadores, labels_anotadores))
    else:
        # Se não passar label, usa o nome da coluna mesmo
        mapa_nomes = {col: col for col in colunas_anotadores}

    # 1. Preparação dos dados
    df_clean = df[colunas_anotadores].dropna().copy()

    for col in colunas_anotadores:
        df_clean[col] = df_clean[col].astype(str).str.upper().str.strip()

    unique_labels = sorted(set(df_clean.values.ravel()))
    label_map = {label: i for i, label in enumerate(unique_labels)}

    df_encoded = df_clean.replace(label_map)
    matrix = df_encoded.values

    resultados = []

    # 2. Cohen's Kappa (Pares Dinâmicos)
    pares = list(combinations(colunas_anotadores, 2))

    cohen_values = []

    for a1, a2 in pares:
        score = cohen_kappa_score(df_encoded[a1], df_encoded[a2])
        cohen_values.append(score)

        # AQUI O AJUSTE: Usa o mapa para pegar o nome bonito
        nome_bonito_a1 = mapa_nomes[a1]
        nome_bonito_a2 = mapa_nomes[a2]

        resultados.append({
            "Métrica": f"Cohen's Kappa ({nome_bonito_a1} vs {nome_bonito_a2})",
            "Valor": score
        })

    # Média do Cohen
    if len(cohen_values) > 0:
        resultados.append({
            "Métrica": "Cohen's Kappa (Média)",
            "Valor": np.mean(cohen_values)
        })

    # 3. Fleiss' Kappa
    agg_data, categories = aggregate_raters(matrix)
    fleiss = fleiss_kappa(agg_data)
    resultados.append({
        "Métrica": "Fleiss' Kappa (Geral)",
        "Valor": fleiss
    })

    # 4. Krippendorff's Alpha
    k_alpha = nominal_krippendorff_alpha(matrix)
    resultados.append({
        "Métrica": "Krippendorff's Alpha (Geral)",
        "Valor": k_alpha
    })

    # 5. Formatação Final
    df_res = pd.DataFrame(resultados)

    def classificar_metricas(row):
        v = row['Valor']
        m = row['Métrica']

        # Se for Krippendorff, usa a escala específica (Krippendorff, 2004)
        if "Krippendorff" in m:
            if v < 0.667: return "Não confiável"
            if v <= 0.800: return "Aceitável"
            if v < 0.900: return "Boa"
            if v > 0.900: return "Excelente"
            return "Confiável"

        # Para Cohen e Fleiss, usa Landis & Koch (1977)
        else:
            if v < 0: return "Pobre"
            if v <= 0.2: return "Leve"
            if v <= 0.4: return "Razoável"
            if v <= 0.6: return "Moderada"
            if v <= 0.8: return "Substancial"
            return "Quase Perfeita"

    df_res['Interpretação'] = df_res.apply(classificar_metricas, axis=1)
    df_res['Valor'] = df_res['Valor'].round(3)

    return df_res

#### Plot on model table

In [ ]:
def plot_tabela_concordancia(df, titulo):
    # Figure configuration
    fig, ax = plt.subplots(figsize=(8, 4)) # Adjustable size (width, height)
    ax.axis('tight')
    ax.axis('off')

    # Create the table
    # cellLoc='center' centers the text
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center', cellLoc='center')

    # Styling
    table.auto_set_font_size(False)
    table.set_fontsize(11) # Font size
    table.scale(1.2, 1.8)  # Adjust spacing (Width, Height of cells)

    # Iterate over cells to apply colors and bold text
    for (row, col), cell in table.get_celld().items():
        cell.set_edgecolor('black') # Simple black border
        cell.set_linewidth(0.5)     # Thin thickness for article

        # Header (Row 0)
        if row == 0:
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor('#333333') # Dark gray/black background

        # Data Rows
        else:
            # Checks if it's a summary line to highlight
            texto_metrica = df.iloc[row-1, 0]
            if "Média" in texto_metrica or "Geral" in texto_metrica:
                cell.set_facecolor('#e6e6e6') # Very light gray for highlighting
                cell.set_text_props(weight='bold') # Bold in final results
            else:
                cell.set_facecolor('white')

    # Plot Title
    plt.title(titulo, weight='bold', size=14, pad=10)

    # Final adjustment and save/show
    plt.tight_layout()
    plt.show()

#### Plote Paper Table

In [ ]:
def renderizar_tabela_imagem(df_consolidado, title):

    df_plot = df_consolidado.copy()

    # --- WIDTH CALCULATION ---
    # Defines a fixed width for 'Model' (e.g., 0.35 = 35% of total width)
    # Divides the remaining (0.65) by the number of other columns
    largura_model = 0.30
    n_outras_cols = len(df_plot.columns) - 1
    largura_outras = (1.0 - largura_model) / n_outras_cols

    # Creates the list of widths in the correct column order
    larguras = []
    for col in df_plot.columns:
        if col == 'Model':
            larguras.append(largura_model)
        else:
            larguras.append(largura_outras)

    # Figure Configuration
    altura_fig = 0.6 * len(df_plot) + 1.2
    fig, ax = plt.subplots(figsize=(14, altura_fig))

    ax.axis('off')

    # Table Creation passing colWidths
    tabela = ax.table(
        cellText=df_plot.values,
        colLabels=df_plot.columns,
        colWidths=larguras, # <--- HERE IS THE MAGIC
        loc='center',
        cellLoc='center',
        bbox=[0, 0, 1, 1]
    )

    # --- Styling ---

    tabela.auto_set_font_size(False)
    tabela.set_fontsize(11)

    for (row, col), cell in tabela.get_celld().items():
        cell.set_height(0.1)

        # Header (Row 0)
        if row == 0:
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor('#40466e')
            cell.set_edgecolor('white')
        else:
            # Zebra Striping
            if row % 2 == 0:
                cell.set_facecolor('#f5f5f5')
            else:
                cell.set_facecolor('white')

            cell.set_edgecolor('#dddddd')
            cell.set_text_props(color='black')

            # Conditional Highlighting (columns 2, 4, 6 are the Interpretation ones)
            if col in [2, 4, 6]:
                try:
                    txt = cell.get_text().get_text()
                    if txt in ['Quase Perfeita', 'Substancial']:
                        cell.set_text_props(weight='bold', color='#2e7d32')
                    elif txt in ['Moderada']:
                        cell.set_text_props(color='#ef6c00')
                    elif txt in ['Leve', 'Pobre']:
                        cell.set_text_props(color='#c62828')
                except:
                    pass

    plt.title(title, fontsize=14, pad=10, weight='bold')
    plt.tight_layout()
    plt.show()

### Execution of similarity metrics of Evalutor A and B.

- results here

#### Calc metric to each combination

In [ ]:
cols = ['RESULT_A_1', 'RESULT_A_2', 'RESULT_A_3']
models_name = ['Qwen3_01', 'Qwen3_02', 'Qwen3_03']
tabela_final_a = gerar_tabela_concordancia("Avaliador A Qwen", final_project_dataset, cols, models_name)

In [ ]:
cols = ['RESULT_B_1', 'RESULT_B_2', 'RESULT_B_3']
models_name = ['Gemma3_01', 'Gemma3_02', 'Gemma3_03']
tabela_final_b = gerar_tabela_concordancia("Avaliador B Gemma", final_project_dataset, cols, models_name)

In [ ]:

cols = ['FINAL_RESULT_A', 'FINAL_RESULT_B']
models_name = ['Qwen3_01', 'Gemma3_01']
tabela_final_a_x_b = gerar_tabela_concordancia("Avaliadores A x B", final_project_dataset, cols, models_name)
# plot_tabela_concordancia(tabela_final_a_x_b, "Avaliadores A x B")

In [ ]:
cols = ['RESULT_JUDGE_A', 'RESULT_JUDGE_B', 'RESULT_JUDGE_C']
models_name = ['M-Prometheus', 'Autoj-6B', 'Phi3-halluc']
judges_concor = gerar_tabela_concordancia("Avaliadores A x B", final_project_dataset, cols, models_name)
# plot_tabela_concordancia(tabela_final_a_x_b, "Avaliadores A x B")

#### Merge Results and plot final table

In [ ]:
tabela_paper = consolidar_resultados([tabela_final_a, tabela_final_b, tabela_final_a_x_b])
renderizar_tabela_imagem(tabela_paper, 'Concordância Avaliadores')

In [ ]:
tabela_paper = consolidar_resultados([judges_concor])
renderizar_tabela_imagem(tabela_paper, 'Concordância Juizes')

### Execution of similarity metrics with ground truth.

- results here

#### Calc metric to each combination

In [ ]:
cols = ['STATUS', 'FINAL_RESULT_A', ]
models_name = ['Ground Truth', 'Qwen3 Consolidado']
tabela_final_a_human = gerar_tabela_concordancia("Avaliador A Qwen", final_project_dataset, cols, models_name)



In [ ]:
cols = ['STATUS', 'FINAL_RESULT_B', ]
models_name = ['Ground Truth', 'Gemma3 Consolidado']
tabela_final_b_human = gerar_tabela_concordancia("Avaliador B Gemma", final_project_dataset, cols, models_name)

In [ ]:
cols = ['STATUS', 'RESULT_JUDGE_A', ]
models_name = ['Ground Truth', '| A∩B + M-Prometheus |']
tabela_final_j1_human = gerar_tabela_concordancia("Avaliador B Gemma", final_project_dataset, cols, models_name)

cols = ['STATUS', 'RESULT_JUDGE_B', ]
models_name = ['Ground Truth', '| A∩B + Autoj-6B |']
tabela_final_j2_human = gerar_tabela_concordancia("Avaliador B Gemma", final_project_dataset, cols, models_name)

cols = ['STATUS', 'RESULT_JUDGE_C', ]
models_name = ['Ground Truth', '| A∩B + Phi3-halluc |']
tabela_final_j3_human = gerar_tabela_concordancia("Avaliador B Gemma", final_project_dataset, cols, models_name)

In [ ]:
cols = ['STATUS', 'FINAL_RESULT', ]
models_name = ['Ground Truth', 'Consenso Final']
final_decision = gerar_tabela_concordancia("", final_project_dataset, cols, models_name)

#### Merge Results and plot final table

In [ ]:
tabela_paper_evaluators_human = consolidar_resultados([
    tabela_final_a_human,
    tabela_final_b_human,
    tabela_final_j1_human,
    tabela_final_j2_human,
    tabela_final_j3_human,
    final_decision
    ])
tabela_paper_evaluators_human = tabela_paper_evaluators_human.drop(columns=['Fleiss Kappa', 'Interp FK'])
renderizar_tabela_imagem(tabela_paper_evaluators_human, 'Concordância Avaliadores x Ground Truth')

### Binary Evaluation Metrics


In [ ]:

# --- Rendering Function  ---
def plotar_tabela_zebrada(df_consolidado, title):
    df_plot = df_consolidado.copy()

    # --- WIDTH CALCULATION ---
    # Defines a fixed width for 'Model' (e.g., 0.35 = 35% of total width)
    largura_model = 0.30
    n_outras_cols = len(df_plot.columns) - 1
    # Avoid division by zero
    largura_outras = (1.0 - largura_model) / n_outras_cols if n_outras_cols > 0 else 0.7

    # Creates the list of widths
    larguras = []
    for col in df_plot.columns:
        if col == 'Model':
            larguras.append(largura_model)
        else:
            larguras.append(largura_outras)

    # Figure Configuration
    altura_fig = 0.5 * len(df_plot) + 1.0
    fig, ax = plt.subplots(figsize=(10, altura_fig))

    ax.axis('off')

    # Table Creation
    tabela = ax.table(
        cellText=df_plot.values,
        colLabels=df_plot.columns,
        colWidths=larguras,
        loc='center',
        cellLoc='center',
        bbox=[0, 0, 1, 1]
    )

    tabela.auto_set_font_size(False)
    tabela.set_fontsize(11)

    for (row, col), cell in tabela.get_celld().items():
        cell.set_height(0.1) # Row height

        # Header (Row 0)
        if row == 0:
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor('#40466e') # Dark Blue
            cell.set_edgecolor('white')

        # Data Rows
        else:
            # Zebra Striping (Alternating colors)
            if row % 2 == 0:
                cell.set_facecolor('#f5f5f5') # Light gray
            else:
                cell.set_facecolor('white')

            cell.set_edgecolor('#dddddd') # Soft border
            cell.set_text_props(color='black')

            # --- Highlighting Logic ---
            try:
                # Get the value from the first column (Model) of this row
                texto_modelo = str(df_plot.iloc[row-1, 0])
                if "Média" in texto_modelo or "Acurácia" in texto_modelo or "Geral" in texto_modelo:
                     cell.set_text_props(weight='bold', color='black')
            except:
                pass

    plt.title(title, fontsize=14, pad=10, weight='bold')
    plt.tight_layout()
    plt.show()

# --- Processing Function ---
def processar_e_avaliar(df_dataset, ground_truth, pred_col, label_exibicao):

    # Cleaning
    df_clean = df_dataset[[ground_truth, pred_col]].dropna()
    y_true = df_clean[ground_truth].astype(str)
    y_pred = df_clean[pred_col].astype(str)

    # --- Step A: Generate Data ---
    report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    df_metrics = pd.DataFrame(report_dict).transpose()

    # Translate and rename
    rename_map = {
        'accuracy': 'Acurácia Geral',
        'macro avg': 'Média Macro',
        'weighted avg': 'Média Ponderada'
    }
    df_metrics = df_metrics.rename(index=rename_map)

    # Reset index and rename to 'Model' to activate width logic
    df_metrics = df_metrics.reset_index().rename(columns={'index': 'Model'})

    # Rounding
    cols_float = ['precision', 'recall', 'f1-score']
    df_metrics[cols_float] = df_metrics[cols_float].round(3)
    df_metrics['support'] = df_metrics['support'].astype(int)

    # Rename final columns to Portuguese
    df_metrics.columns = ['Model', 'Precisão', 'Recall', 'F1-Score', 'Suporte']

    # --- Step B: Plot Table (WITH NEW NAME) ---
    plotar_tabela_zebrada(df_metrics, f"Métricas: {label_exibicao}")

    # --- Step C: Plot Confusion Matrix ---
    classes = sorted(list(set(y_true.unique()) | set(y_pred.unique())))
    cm = confusion_matrix(y_true, y_pred, labels=classes)

    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Matriz de Confusão: {label_exibicao}')
    plt.ylabel('Real')
    plt.xlabel('Predito')
    plt.tight_layout()
    plt.show()

In [ ]:
# Configuration dictionary mapping internal result column names to display labels
configuracao_execucao = {
    'FINAL_RESULT_A': 'Qwen3-4B-Instruct',
    'FINAL_RESULT_B': 'Gemma-3-4b-it',
    'RESULT_JUDGE_A': 'M-Prometheus-3B',
    'RESULT_JUDGE_B': 'Autoj-bilingual-6b',
    'RESULT_JUDGE_C': 'Phi3-hallucination-judge',
    'FINAL_RESULT':   'Consenso Final'
}

# Iterate through each configured model/judge and process its results
for coluna, label in configuracao_execucao.items():
    try:
        # Call the function to calculate and plot binary evaluation metrics (classification report and confusion matrix)
        # 'STATUS' is used as the ground truth, 'coluna' is the prediction column, and 'label' is for display purposes
        processar_e_avaliar(final_project_dataset, 'STATUS', coluna, label)
    except Exception as e:
        # Catch and print any errors that occur during processing for a specific column
        print(f"Error processing {coluna}: {e}")